Импорты:

In [1]:
import numpy as np
import pandas as pd
import asyncio
import struct
import nest_asyncio
import bleak
from bleak import BleakScanner, BleakClient
from bleak.exc import BleakError
nest_asyncio.apply()
import re
from tqdm import tqdm
from queue import Queue

In [2]:
class BleakClientAssistant:
    def __init__(self, device, timeout=60):
        self.device = device
        self.timeout = timeout
        self.loop = asyncio.get_event_loop()
        self.client = None
        self.hk = None
        self.lk = None

    async def connect(self):
        self.client = BleakClient(self.device, timeout=self.timeout, cache_timeout=60)
        try:
            with tqdm(total=100, desc=f"Connecting and sending data on {self.device.name}:") as pbar:
                success = await self.client.connect()
                for i in range(100):
                    pbar.update(1)
                    await asyncio.sleep(0.1)
                if success:
                    await asyncio.sleep(1)  
                    await self.client.start_notify(14, self.callback)
                else:
                    print("Failed to connect to device")
        except BleakError as e:
            print(f"Could not connect to device: {e}")
            for i in range(10):
                print(f"Retrying connect {i+1}/10...")
                try:
                    with tqdm(total=100, desc=f"Connecting and sending data on {self.device.name}:") as pbar:
                        success = await self.client.connect()
                        for i in range(100):
                            pbar.update(1)
                            await asyncio.sleep(0.1)
                        if success:
                            await asyncio.sleep(1)  
                            await self.client.start_notify(14, self.callback)
                            return
                        else:
                            print("Failed to connect to device")
                except BleakError as e:
                    print(f"Could not connect to device: {e}")
            retry = input("Would you like to retry connect? (y/n) ")
            if retry.lower() == "y":
                await self.connect()
            else:
                return
    
    async def disconnect(self):
        await self.client.stop_notify(14)
        await self.client.disconnect()
        
    async def send_message(self, message):
        try:
            await self.client.write_gatt_char(12, message)
            print(f'WRITE: {message} on TX_char')
            await asyncio.sleep(1)
        except Exception as e:
            print(f"Error in send message (client): {e}")
    async def callback(self, sender: int, data: bytearray):
        try:
            response = data.decode().strip()
            print(f'Received notification from RX_char: {response}')
            match = re.search(r"HK:1:(\d+),LK:1:(\d+)", response)
            if match:
                self.hk = int(match.group(1))
                self.lk = int(match.group(2))
        except Exception as e:
            print(f"Error in callback (client): {e}")
    def run(self, message):
        try:
            self.loop.run_until_complete(self.connect())
            self.loop.run_until_complete(self.send_message(message))
            self.loop.run_until_complete(self.disconnect())
            return self.hk, self.lk
        except Exception as e:
            print(f"Error in run (client): {e}")

In [3]:
def adv_decrypt(data):
    # Расшифровка уровня топлива
    oil_level_raw = int.from_bytes(data[1:3], byteorder='little', signed=False)
    print(f"Oil level: {oil_level_raw}")

    # Расшифровка напряжения встроенной батареи
    battery_voltage_raw = int.from_bytes(data[3:4], byteorder='little', signed=False)
    battery_voltage = battery_voltage_raw / 10.0
    print(f"Battery voltage: {battery_voltage}V")

    # Расшифровка температуры
    TD_temp_raw = int.from_bytes(data[4:5], byteorder='little', signed=True)
    print(f"Temperature: {TD_temp_raw}℃")

    return oil_level_raw, battery_voltage, TD_temp_raw

In [4]:
class DataCollector:
    def __init__(self, start_serial, end_serial):
        self.start_serial = start_serial
        self.end_serial = end_serial
        self.device_names = ["TD_" + str(i) for i in range(start_serial, end_serial+1)]
        self.start_len = len(self.device_names)
        self.df = pd.DataFrame({'Name': self.device_names, 
                                'MAC': [None] * len(self.device_names),
                                'RSSI': [None] * len(self.device_names),
                                'Oil Level' : [None] * len(self.device_names),
                                'Temperature': [None] * len(self.device_names),
                                'Battery Voltage': [None] * len(self.device_names),
                                'HK': [None] * len(self.device_names),
                                'LK': [None] * len(self.device_names)})
        
    def update_MAC(self, name, address):
        index = self.df.index[self.df['Name'] == name][0]
       # if self.df.at[index, 'MAC'] is None:   
        self.df.at[index, 'MAC'] = address
            
    def update_RSSI(self, name, RSSI):
        index = self.df.index[self.df['Name'] == name][0]
       # if self.df.at[index, 'RSSI'] is None:   
        self.df.at[index, 'RSSI'] = RSSI
            
    def update_oil_level(self, name, oil_level_raw):
        index = self.df.index[self.df['Name'] == name][0]
       # if self.df.at[index, 'Oil Level'] is None:   
        self.df.at[index, 'Oil Level'] = oil_level_raw
            
    def update_temp(self, name, TD_temp_raw):
        index = self.df.index[self.df['Name'] == name][0]
       # if self.df.at[index, 'Temperature'] is None:   
        self.df.at[index, 'Temperature'] = TD_temp_raw
            
    def update_battery_voltage(self, name, battery_voltage):
        index = self.df.index[self.df['Name'] == name][0]
        # if self.df.at[index, 'Battery Voltage'] is None:   
        self.df.at[index, 'Battery Voltage'] = battery_voltage
            
    def update_HK(self, name, hk):
        index = self.df.index[self.df['Name'] == name][0]
       # if self.df.at[index, 'HK'] is None:   
        self.df.at[index, 'HK'] = hk
            
    def update_LK(self, name, lk):
        index = self.df.index[self.df['Name'] == name][0]
       # if self.df.at[index, 'LK'] is None:   
        self.df.at[index, 'LK'] = lk
            
    def get_dataframe(self):
        return self.df

In [5]:
class MyScanner:
    def __init__(self, timeout, start_serial, end_serial):
        self._scanner = BleakScanner(detection_callback=self.detection_callback)
        self.scanning = asyncio.Event()
        self.timeout = timeout
        self.dc = DataCollector(start_serial, end_serial)
        self.queue_devices_to_connect = Queue()

    def detection_callback(self, device, advertisement_data):
        try:
            if device.name in self.dc.device_names: 
                
                
                self.queue_devices_to_connect.put(device)
            
                self.dc.device_names.remove(device.name)
            
                print(f'found device with name: {device.name} {self.dc.start_len-len(self.dc.device_names)}/{self.dc.start_len}')
                self.dc.update_MAC(device.name, device.address)
                self.dc.update_RSSI(device.name, advertisement_data.rssi)
            
                oil_level_raw, battery_voltage, TD_temp_raw = adv_decrypt(advertisement_data.manufacturer_data[3862])
                self.dc.update_oil_level(device.name, oil_level_raw)
                self.dc.update_battery_voltage(device.name, battery_voltage)
                self.dc.update_temp(device.name, TD_temp_raw)
        except Exception as e:
            print(f"Error in callback (scanner): {e}")
    async def run(self):
        self._scanner = BleakScanner(detection_callback=self.detection_callback)
        print(f'\t\tStarted scanning with {self.timeout} seconds timeout...')
        await self._scanner.start()
        self.scanning.set()
        end_time = loop.time() + self.timeout
        while self.scanning.is_set():
            if loop.time() > end_time:
                self.scanning.clear()
                await self._scanner.stop()
                print(f"\t\tScan has timed out so we terminate.")
                if len(self.dc.device_names) == 0:
                    print('All devices have been found.')
                else:
                    print(f"Can't founded devices ({len(self.dc.device_names)*100/self.dc.start_len:.2f}%) : {self.dc.device_names}")
                    while True:
                        retry = input("Would you like to retry search? (y/n) ")
                        if retry.lower() == "y":
                            change_timeout = input('Would you like to change timeout? (y/n) ')
                            if change_timeout.lower() == "y":
                                self.timeout = int(input('Timeout seconds (int): '))
                                await self.run()
                                break
                            else:
                                await self.run()
                                break
                        elif retry.lower() == "n":
                            return
                        else:
                            print("Invalid input, please enter 'y' or 'n'.")
                await asyncio.wait(0.01)
            else:
                continue
    def get_dataframe(self):
        return self.dc.get_dataframe()
    
    def queue_connection(self):
        print(f'\t\tStarted connecting and sending data for queue of devices...')
        while not self.queue_devices_to_connect.empty():
            print('Devices in queue:', self.queue_devices_to_connect.queue)
            device = self.queue_devices_to_connect.get()
            assistant = BleakClientAssistant(device)
            hk, lk = assistant.run(b"GA\r")
            self.dc.update_HK(device.name, hk)
            self.dc.update_LK(device.name, lk)

In [6]:
%%time
my_scanner = MyScanner(timeout=10, start_serial=383448, end_serial=383450)
loop = asyncio.get_event_loop()
loop.run_until_complete(my_scanner.run())
my_scanner.queue_connection()

		Started scanning with 10 seconds timeout...
		Scan has timed out so we terminate.
Can't founded devices (100.00%) : ['TD_383448', 'TD_383449', 'TD_383450']
Would you like to retry search? (y/n) y
Would you like to change timeout? (y/n) n
		Started scanning with 10 seconds timeout...
		Scan has timed out so we terminate.
Can't founded devices (100.00%) : ['TD_383448', 'TD_383449', 'TD_383450']
Would you like to retry search? (y/n) n


TypeError: 'float' object is not iterable

In [8]:
loop.is_running()

True

In [19]:
my_scanner.get_dataframe()

,Name,MAC,RSSI,Oil Level,Temperature,Battery Voltage,HK,LK
0,TD_383448,CA:17:37:73:61:98,-50,4095,24,3.7,None,None
1,TD_383449,E2:F1:F3:8C:E1:45,-54,1,24,3.7,None,None
2,TD_383450,None,None,None,None,None,None,None
